In [1]:
import json
import itertools
import re
import os
from pathlib import Path

In [2]:
!pip3 install wikipedia-api

In [3]:
!pip install openai


In [5]:
from openai import OpenAI

In [6]:
client = OpenAI()

In [7]:
data_dir = Path("base_model/data/tree_batch_one_data")

In [8]:
files_to_load = [
    "final_batch_enriched_tree_culture.json",
    "final_batch_enriched_tree_politics.json",
    "final_batch_enriched_tree_science.json",
    "final_batch_enriched_tree_social.json",
    "final_batch_enriched_tree_sports.json",
    "final_batch_enriched_tree_wellness.json"
]
# Load all files into a dictionary
enriched_data = {}
for file_name in files_to_load:
    file_path = data_dir / file_name
    with open(file_path, 'r') as f:
        enriched_data[file_name] = json.load(f)
    print(f"Loaded file: {file_name}")


Loaded file: final_batch_enriched_tree_culture.json
Loaded file: final_batch_enriched_tree_politics.json
Loaded file: final_batch_enriched_tree_science.json
Loaded file: final_batch_enriched_tree_social.json
Loaded file: final_batch_enriched_tree_sports.json
Loaded file: final_batch_enriched_tree_wellness.json


In [9]:
files_to_load = [
    "final_batch_enriched_tree_culture.json",
    "final_batch_enriched_tree_politics.json",
    "final_batch_enriched_tree_science.json",
    "final_batch_enriched_tree_social.json",
    "final_batch_enriched_tree_sports.json",
    "final_batch_enriched_tree_wellness.json"
]
# Load all files into a dictionary
enriched_data = {}
for file_name in files_to_load:
    file_path = data_dir / file_name
    with open(file_path, 'r') as f:
        enriched_data[file_name] = json.load(f)
    print(f"Loaded file: {file_name}")


Loaded file: final_batch_enriched_tree_culture.json
Loaded file: final_batch_enriched_tree_politics.json
Loaded file: final_batch_enriched_tree_science.json
Loaded file: final_batch_enriched_tree_social.json
Loaded file: final_batch_enriched_tree_sports.json
Loaded file: final_batch_enriched_tree_wellness.json


In [10]:
def traverse_tree_and_extract_data(node, results=None):
    if results is None:
        results = []

    # Print the current node being processed
    print(f"Processing node: {node.get('name', 'Unnamed Node')}")

    # Extract 'api_query_child' content if present
    api_query_child = node.get("api_query_child", {})

    # If `api_query_child` has content, append it to results and print
    if api_query_child:
        print(f"Found 'api_query_child' in node: {node.get('name', '')}")
        results.append({
            "node_name": node.get("name", ""),
            "api_query_child": api_query_child
        })

    # Recurse through children
    for child_name, child_node in node.get("children", {}).items():
        traverse_tree_and_extract_data(child_node, results)

    return results


In [11]:
# Dictionary to store extracted data from each file
extracted_data = {}

for file_name, tree_data in enriched_data.items():
    print(f"\nExtracting data from: {file_name}")
    extracted_nodes = traverse_tree_and_extract_data(tree_data)
    extracted_data[file_name] = extracted_nodes
    print(f"Extracted {len(extracted_nodes)} nodes with 'api_query_child' from {file_name}")



Extracting data from: final_batch_enriched_tree_culture.json
Processing node: Root
Found 'api_query_child' in node: Root
Processing node: details
Found 'api_query_child' in node: details
Processing node: culture
Found 'api_query_child' in node: culture
Processing node: details
Processing node: Pop Culture
Found 'api_query_child' in node: Pop Culture
Processing node: details
Processing node: Music
Found 'api_query_child' in node: Music
Processing node: details
Processing node: Hip-Hop
Found 'api_query_child' in node: Hip-Hop
Processing node: details
Processing node: Pop
Processing node: details
Processing node: K-Pop
Found 'api_query_child' in node: K-Pop
Processing node: details
Processing node: EDM
Found 'api_query_child' in node: EDM
Processing node: details
Processing node: Movies and TV Shows
Found 'api_query_child' in node: Movies and TV Shows
Processing node: details
Processing node: streaming platforms
Found 'api_query_child' in node: streaming platforms
Processing node: detail

In [12]:
# Example: Print summaries for generating training prompts
for file_name, nodes in extracted_data.items():
    print(f"\n--- Generating training data for {file_name} ---")
    for i, node in enumerate(nodes):
        node_name = node["node_name"]
        api_query_child = node["api_query_child"]
        summary = api_query_child.get("original_query", {}).get("summary", "")

        # Display some of the extracted content
        print(f"Node {i+1}: {node_name}")
        print(f"  Summary (first 100 chars): {summary[:100]}...")
        print(f"  Total sections: {len(api_query_child.get('original_query', {}).get('sections', []))}")
        print(f"  Links count: {len(api_query_child.get('original_query', {}).get('links', []))}")
        print(f"  Categories count: {len(api_query_child.get('original_query', {}).get('categories', []))}")

        # Add a break to avoid overwhelming output
        if i >= 5:  # Adjust this number to print more or fewer examples
            print("...")
            break



--- Generating training data for final_batch_enriched_tree_culture.json ---
Node 1: Root
  Summary (first 100 chars): In vascular plants, the roots are the organs of a plant that are modified to provide anchorage for t...
  Total sections: 17
  Links count: 10
  Categories count: 9
Node 2: details
  Summary (first 100 chars): Detail(s) or The Detail(s) may refer to:...
  Total sections: 4
  Links count: 10
  Categories count: 4
Node 3: culture
  Summary (first 100 chars): Culture ( KUL-chər) is a concept that encompasses the social behavior, institutions, and norms found...
  Total sections: 13
  Links count: 10
  Categories count: 23
Node 4: Pop Culture
  Summary (first 100 chars): Popular culture (also called pop culture or mass culture) is generally recognized by members of a so...
  Total sections: 10
  Links count: 10
  Categories count: 24
Node 5: Music
  Summary (first 100 chars): Music is the arrangement of sound to create some combination of form, harmony, melody, rhythm, or 

In [13]:
def traverse_tree_and_extract_data(node, results=None, limit=10):
    if results is None:
        results = []

    # Check the limit of printed nodes
    if len(results) >= limit:
        return results

    # Process the current node
    node_name = node.get("name", "Unnamed Node")
    print(f"Processing node: {node_name}")

    # Extract 'api_query_child' content if present
    api_query_child = node.get("api_query_child", {})
    if api_query_child:
        # Collect item names in `api_query_child`
        item_names = list(api_query_child.keys())
        print(f"Node: {node_name}")
        print(f"api_query_node_items_list: {item_names}")

        # Append to results with the collected information
        results.append({
            "node_name": node_name,
            "api_query_node_items_list": item_names
        })

    # Recurse through children
    for child_name, child_node in node.get("children", {}).items():
        if len(results) >= limit:
            break
        traverse_tree_and_extract_data(child_node, results, limit)

    return results


In [14]:
# Dictionary to store extracted data from each file
extracted_data = {}

for file_name, tree_data in enriched_data.items():
    print(f"\nExtracting data from: {file_name}")
    extracted_nodes = traverse_tree_and_extract_data(tree_data)
    extracted_data[file_name] = extracted_nodes
    print(f"Extracted {len(extracted_nodes)} nodes with 'api_query_child' from {file_name}")


Extracting data from: final_batch_enriched_tree_culture.json
Processing node: Root
Node: Root
api_query_node_items_list: ['original_query']
Processing node: details
Node: details
api_query_node_items_list: ['original_query']
Processing node: culture
Node: culture
api_query_node_items_list: ['original_query']
Processing node: details
Processing node: Pop Culture
Node: Pop Culture
api_query_node_items_list: ['original_query', 'Culture', 'Pop']
Processing node: details
Processing node: Music
Node: Music
api_query_node_items_list: ['original_query']
Processing node: details
Processing node: Hip-Hop
Node: Hip-Hop
api_query_node_items_list: ['original_query']
Processing node: details
Processing node: Pop
Processing node: details
Processing node: K-Pop
Node: K-Pop
api_query_node_items_list: ['original_query']
Processing node: details
Processing node: EDM
Node: EDM
api_query_node_items_list: ['original_query']
Processing node: details
Processing node: Movies and TV Shows
Node: Movies and TV S

In [15]:
print(extracted_data)

{'final_batch_enriched_tree_culture.json': [{'node_name': 'Root', 'api_query_node_items_list': ['original_query']}, {'node_name': 'details', 'api_query_node_items_list': ['original_query']}, {'node_name': 'culture', 'api_query_node_items_list': ['original_query']}, {'node_name': 'Pop Culture', 'api_query_node_items_list': ['original_query', 'Culture', 'Pop']}, {'node_name': 'Music', 'api_query_node_items_list': ['original_query']}, {'node_name': 'Hip-Hop', 'api_query_node_items_list': ['original_query']}, {'node_name': 'K-Pop', 'api_query_node_items_list': ['original_query']}, {'node_name': 'EDM', 'api_query_node_items_list': ['original_query']}, {'node_name': 'Movies and TV Shows', 'api_query_node_items_list': ['TV Shows', 'Movies', 'TV', 'Shows']}, {'node_name': 'streaming platforms', 'api_query_node_items_list': ['platforms', 'streaming']}], 'final_batch_enriched_tree_politics.json': [{'node_name': 'Root', 'api_query_node_items_list': ['original_query']}, {'node_name': 'details', 'a

In [16]:
# Define a global variable to hold the extracted API query data
extracted_api_query_data = {}

def print_detailed_api_query_child(node, limit=10):
    """
    Traverse the enriched tree to print and save detailed contents of each `api_query_child` in a structured format.
    Only prints and saves up to `limit` items for testing.
    """
    global extracted_api_query_data
    printed_count = 0

    def traverse_and_print(node, level=0):
        nonlocal printed_count
        if printed_count >= limit:
            return

        # Initialize an entry for the current node in the global dictionary
        node_name = node.get('name', 'Unnamed Node')
        if node_name not in extracted_api_query_data:
            extracted_api_query_data[node_name] = []

        # If there's an `api_query_child`, gather details and store in the global dictionary
        if 'api_query_child' in node:
            api_query_node_details = []
            print(f"\nNode: {node_name}")
            print("api_query_node_items: [")

            for key, value in node['api_query_child'].items():
                # Collect details for each item in `api_query_child`
                item_details = {
                    "item_name": key,
                    "page_title": value.get('page_title', 'N/A'),
                    "summary": value.get('summary', 'N/A')[:100] + "...",  # Limit summary for brevity
                    "sections": [section.get('title', 'No Title') for section in value.get('sections', [])[:3]],
                    "links": value.get('links', [])[:3],
                    "categories": value.get('categories', [])[:3],
                    "category_submembers": [member.get('title', 'No Title') for member in value.get('category_sub_members', [])[:3]],
                }
                
                # Print each item's details
                print(f"\n  Item Name: {item_details['item_name']}")
                print(f"    page_title: {item_details['page_title']}")
                print(f"    summary: {item_details['summary']}")
                print("    sections:", ", ".join(item_details['sections']) + "...")
                print("    links:", ", ".join(item_details['links']) + "...")
                print("    categories:", ", ".join(item_details['categories']) + "...")
                print("    category_submembers:", ", ".join(item_details['category_submembers']) + "...")

                # Append the structured item details to the list for this node
                api_query_node_details.append(item_details)
            print("]")

            # Store each node's details in the global dictionary
            extracted_api_query_data[node_name].extend(api_query_node_details)

            printed_count += 1

        # Recurse through children
        if 'children' in node:
            for child_name, child_node in node['children'].items():
                traverse_and_print(child_node, level + 1)

    # Start traversal from the root node
    traverse_and_print(node)


In [17]:
for file_name, tree_data in enriched_data.items():
    print(f"\n--- Processing file: {file_name} ---")
    print_detailed_api_query_child(tree_data, limit=40)


--- Processing file: final_batch_enriched_tree_culture.json ---

Node: Root
api_query_node_items: [

  Item Name: original_query
    page_title: Root
    summary: In vascular plants, the roots are the organs of a plant that are modified to provide anchorage for t...
    sections: Function, Anatomy, Root system architecture (RSA)...
    links: ABC model of flower development, Absorption of water, Acacia erioloba...
    categories: Category:All articles needing additional references, Category:All articles with unsourced statements, Category:Articles needing additional references from March 2010...
    category_submembers: Category:All articles needing additional references, Category:All articles with unsourced statements, Category:Articles needing additional references from March 2010...
]

Node: details
api_query_node_items: [

  Item Name: original_query
    page_title: Detail
    summary: Detail(s) or The Detail(s) may refer to:...
    sections: Film and television, Music, Periodical

In [18]:
#printing out the first ten items to see what we have: 
def print_first_ten_items(data):
    """
    Print the first ten items from the `extracted_api_query_data` dictionary.
    """
    item_count = 0
    for node, items in data.items():
        print(f"\nNode: {node}")
        
        for item in items:
            # Print item details
            print(f"\n  Item Name: {item.get('item_name', 'Unnamed Item')}")
            print(f"    Page Title: {item.get('page_title', 'N/A')}")
            print(f"    Summary: {item.get('summary', 'N/A')}")
            print(f"    Sections: {', '.join(item.get('sections', [])[:5])}...")
            print(f"    Links: {', '.join(item.get('links', [])[:5])}...")
            print(f"    Categories: {', '.join(item.get('categories', [])[:5])}...")
            print(f"    Category Submembers: {', '.join(item.get('category_submembers', [])[:5])}...")

            item_count += 1
            if item_count >= 50:
                return  # Stop after printing 10 items

# Call the function with `extracted_api_query_data`
print_first_ten_items(extracted_api_query_data)



Node: Root

  Item Name: original_query
    Page Title: Root
    Summary: In vascular plants, the roots are the organs of a plant that are modified to provide anchorage for t...
    Sections: Function, Anatomy, Root system architecture (RSA)...
    Links: ABC model of flower development, Absorption of water, Acacia erioloba...
    Categories: Category:All articles needing additional references, Category:All articles with unsourced statements, Category:Articles needing additional references from March 2010...
    Category Submembers: Category:All articles needing additional references, Category:All articles with unsourced statements, Category:Articles needing additional references from March 2010...

  Item Name: original_query
    Page Title: Root
    Summary: In vascular plants, the roots are the organs of a plant that are modified to provide anchorage for t...
    Sections: Function, Anatomy, Root system architecture (RSA)...
    Links: ABC model of flower development, Absorption of

In [19]:
# Function to create prompt structure for each item in extracted_api_query_data
def create_prompts_from_extracted_data(data):
    """
    Generate a prompt structure for each item in extracted_api_query_data.
    Ensure uniqueness and better formatting.
    """
    prompts = []
    seen_combinations = set()  # To track unique node and item combinations

    for node_name, items in data.items():
        for i, item in enumerate(items):
            # Extract details from the item
            api_query_child_name = item.get("item_name", f"Unnamed Item {i}")
            page_title = item.get("page_title", "N/A")
            summary = item.get("summary", "N/A")
            sections = ", ".join(item.get("sections", [])[:5])  # Limited to 5 for clarity
            links = ", ".join(item.get("links", [])[:5])
            category_submembers = ", ".join(item.get("category_submembers", [])[:5])

            # Ensure the combination of node_name and api_query_child_name is unique
            if (node_name, api_query_child_name) in seen_combinations:
                continue
            seen_combinations.add((node_name, api_query_child_name))

            # Create the prompt structure with unique details per item
            subtopics = ", ".join(filter(None, [sections, links, category_submembers]))
            prompt_data = {
                "node_name": node_name,
                "api_query_child_name": api_query_child_name,
                "prompt_extracted": {
                    f"these are details about the following subject: {node_name}": {
                        "Topic": page_title,
                        "Summary": summary,
                        "Subtopics": subtopics
                    }
                }
            }

            # Append the created prompt to the list
            prompts.append(prompt_data)

    return prompts 

# Count the total items in extracted_api_query_data for verification
total_items = sum(len(items) for items in extracted_api_query_data.values())
print(f"Expected total number of items in extracted_api_query_data: {total_items}")

# Call the function and store the prompts globally
generated_prompts = create_prompts_from_extracted_data(extracted_api_query_data)

# Check if the generated prompt count matches the expected item count
print(f"Generated prompt count: {len(generated_prompts)}")
if len(generated_prompts) == total_items:
    print("All items in extracted_api_query_data were processed.")
else:
    print("Warning: Some items may not have been processed correctly.")

# Print the first 10 unique prompts to verify the structure
for prompt in generated_prompts[:20]:
    print(json.dumps(prompt, indent=2))


Expected total number of items in extracted_api_query_data: 211
Generated prompt count: 201
{
  "node_name": "Root",
  "api_query_child_name": "original_query",
  "prompt_extracted": {
    "these are details about the following subject: Root": {
      "Topic": "Root",
      "Summary": "In vascular plants, the roots are the organs of a plant that are modified to provide anchorage for t...",
      "Subtopics": "Function, Anatomy, Root system architecture (RSA), ABC model of flower development, Absorption of water, Acacia erioloba, Category:All articles needing additional references, Category:All articles with unsourced statements, Category:Articles needing additional references from March 2010"
    }
  }
}
{
  "node_name": "details",
  "api_query_child_name": "original_query",
  "prompt_extracted": {
    "these are details about the following subject: details": {
      "Topic": "Detail",
      "Summary": "Detail(s) or The Detail(s) may refer to:...",
      "Subtopics": "Film and televisi

In [20]:
# import openai
# import time
# import random
# import json

# # Initialize OpenAI client
# client = openai.OpenAI()

# # Define global constants and variables
# output_file = "squad_names_training_data.jsonl"  # Incremental saving
# batch_size = 800  # Moderate batch size for API requests
# names_per_prompt = 20_000  # Total squad names to generate per prompt
# suffix_list = [  # Diverse suffixes for prompt variation
#     "Include as many types as possible.",
#     "Think creatively and avoid repetition.",
#     "Consider various types of people groups.",
#     "Cover a diverse range of interests.",
#     "Use different community styles and languages.",
#     "Be inclusive of all kinds of interests.",
#     "Generate some unexpected group names.",
#     "Think of unique groups and subcultures.",
#     "Avoid generic terms; make names memorable.",
#     "Include academic, recreational, and cultural squads.",
#     "Consider groups with both local and global appeal.",
#     "Use both formal and informal styles of naming.",
#     "Include both niche and mainstream interests.",
#     "Think of names that appeal to different ages.",
#     "Include names that would fit specific settings.",
#     "Focus on creative and new community names.",
#     "Generate group names that sound modern.",
#     "Use some names that include numbers or symbols.",
#     "Consider groups based on new or emerging trends.",
#     "Think of groups that connect online and offline.",
#     "Suggest names for both large and small groups.",
#     "Consider local, national, and global communities.",
#     "Think about groups with seasonal interests.",
#     "Use names that reflect popular events or festivals.",
#     "Consider student clubs and academic groups.",
#     "Include names for different professional groups.",
#     "Think of environmental, tech, and arts-based groups.",
#     "Use quirky and fun names for casual gatherings.",
#     "Include groups for both indoor and outdoor activities.",
#     "Think of names that involve social media trends.",
#     "Consider groups with culinary or travel interests.",
#     "Use names that appeal to niche online communities.",
#     "Include groups with philosophical or wellness themes.",
#     "Consider groups that might meet for specific hobbies.",
#     "Think about both active and relaxed group settings.",
#     "Use names that include popular slang or phrases.",
#     "Incorporate recent trends in naming conventions.",
#     "Generate some groups based on personality types.",
#     "Consider historical and futuristic group themes.",
#     "Include groups related to lifestyle and mindfulness.",
#     "Think of names for book clubs and media fans.",
#     "Include groups for gaming, technology, and science.",
#     "Think of groups for pet lovers and animal advocates.",
#     "Consider interest groups for different age groups.",
#     "Use names that might appeal to specific cultures.",
#     "Consider names that evoke a sense of adventure.",
#     "Include both serious and humorous group names.",
#     "Think about groups for fitness and sports fans.",
#     "Consider names for people interested in learning.",
#     "Include names for groups that focus on relaxation.",
# ]


# def save_batch_to_jsonl(data, filename):
#     """
#     Save a batch of squad name data to a JSONL file incrementally.
#     """
#     with open(filename, "a") as f:
#         for entry in data:
#             jsonl_entry = {
#                 "prompt": entry["prompt"],
#                 "completion": ", ".join(entry["squad_names"]),
#             }
#             f.write(json.dumps(jsonl_entry) + "\n")
#     print(f"Batch of {len(data)} squad names saved to {filename}")


# def generate_squad_names_from_summary(summary_text, target_count=names_per_prompt):
#     """
#     Generate squad names using the OpenAI client based on a given summary text.
#     """
#     generated_squads = set()
#     while len(generated_squads) < target_count:
#         random_suffix = random.choice(suffix_list)
#         try:
#             completion = client.chat.completions.create(
#                 model="gpt-4",
#                 messages=[
#                     {"role": "system", "content": "You are a helpful assistant for creating squad names."},
#                     {
#                         "role": "user",
#                         "content": (
#                             f"Based on the following description, suggest up to {batch_size} unique names for interest groups "
#                             f"or squads of people with shared interests:\n\n{summary_text}\n\n"
#                             "Generate names that include a variety of interests and naturally occurring groups, "
#                             "such as 'Athletes Squad,' 'Music Lovers Community,' 'Debate Club Enthusiasts,' 'Environmental Activists,' "
#                             "and 'Tech Innovators.' Be creative and make sure each name sounds like a natural group. " + random_suffix
#                         ),
#                     },
#                 ],
#             )
#             response_text = completion.choices[0].message.content.strip()
#             squad_names = response_text.split("\n")
#             generated_squads.update([name.strip() for name in squad_names if name.strip()])
#             print(f"Generated {len(generated_squads)} unique squads so far.")
#             time.sleep(1)  # Respect API limits

#         except openai.error.OpenAIError as e:
#             print(f"Error generating squad names: {e}")
#             break

#     return list(generated_squads)[:target_count]


# def generate_squads_from_prompts(prompts):
#     """
#     Generate squad names for each prompt in the provided prompts list using OpenAI API,
#     and save results incrementally.
#     """
#     total_generated = 0

#     for prompt in prompts:
#         node_name = prompt.get("node_name", "Unknown Node")
#         api_query_child_name = prompt.get("api_query_child_name", "Unknown Query")
#         prompt_content = prompt["prompt_extracted"]

#         summary_text = (
#             f"{prompt_content['these are details about the following subject: ' + node_name]['Summary']}\n\n"
#             f"Additional details:\n"
#             f"Topic: {prompt_content['these are details about the following subject: ' + node_name]['Topic']}\n"
#             f"Subtopics: {prompt_content['these are details about the following subject: ' + node_name]['Subtopics']}"
#         )

#         print(f"Generating squad names for {node_name} - {api_query_child_name}")

#         squad_names = generate_squad_names_from_summary(summary_text)

#         batch_entry = {
#             "node_name": node_name,
#             "api_query_child_name": api_query_child_name,
#             "prompt": summary_text,
#             "squad_names": squad_names,
#         }
#         total_generated += len(squad_names)

#         # Save the batch incrementally
#         save_batch_to_jsonl([batch_entry], output_file)

#     print("\nSquad name generation completed.")
#     print(f"Total squad names generated: {total_generated}")


# # Example run with generated prompts
# generate_squads_from_prompts(generated_prompts)


In [21]:
print(generated_prompts[:10])

[{'node_name': 'Root', 'api_query_child_name': 'original_query', 'prompt_extracted': {'these are details about the following subject: Root': {'Topic': 'Root', 'Summary': 'In vascular plants, the roots are the organs of a plant that are modified to provide anchorage for t...', 'Subtopics': 'Function, Anatomy, Root system architecture (RSA), ABC model of flower development, Absorption of water, Acacia erioloba, Category:All articles needing additional references, Category:All articles with unsourced statements, Category:Articles needing additional references from March 2010'}}}, {'node_name': 'details', 'api_query_child_name': 'original_query', 'prompt_extracted': {'these are details about the following subject: details': {'Topic': 'Detail', 'Summary': 'Detail(s) or The Detail(s) may refer to:...', 'Subtopics': 'Film and television, Music, Periodicals, Auto detailing, DETAIL (professional journal), Detail (record producer), Category:All article disambiguation pages, Category:All disambig

In [22]:
# import openai
# import time
# import random
# import json

# # Initialize OpenAI client
# client = openai.OpenAI()

# # Define global constants and variables
# output_file = "squad_names_training_data.jsonl"  # Incremental saving
# batch_size = 500  # Moderate batch size for API requests
# names_per_prompt = 2500  # Total squad names to generate per prompt
# suffix_list = [  # Diverse suffixes for prompt variation
#     "Include as many types as possible.",
#     "Think creatively and avoid repetition.",
#     "Consider various types of people groups.",
#     "Cover a diverse range of interests.",
#     "Use different community styles and languages.",
#     "Be inclusive of all kinds of interests.",
#     "Generate some unexpected group names.",
#     "Think of unique groups and subcultures.",
#     "Avoid generic terms; make names memorable.",
#     "Include academic, recreational, and cultural squads.",
#     "Consider groups with both local and global appeal.",
#     "Use both formal and informal styles of naming.",
#     "Include both niche and mainstream interests.",
#     "Think of names that appeal to different ages.",
#     "Include names that would fit specific settings.",
#     "Focus on creative and new community names.",
#     "Generate group names that sound modern.",
#     "Use some names that include numbers or symbols.",
#     "Consider groups based on new or emerging trends.",
#     "Think of groups that connect online and offline.",
#     "Suggest names for both large and small groups.",
#     "Consider local, national, and global communities.",
#     "Think about groups with seasonal interests.",
#     "Use names that reflect popular events or festivals.",
#     "Consider student clubs and academic groups.",
#     "Include names for different professional groups.",
#     "Think of environmental, tech, and arts-based groups.",
#     "Use quirky and fun names for casual gatherings.",
#     "Include groups for both indoor and outdoor activities.",
#     "Think of names that involve social media trends.",
#     "Consider groups with culinary or travel interests.",
#     "Use names that appeal to niche online communities.",
#     "Include groups with philosophical or wellness themes.",
#     "Consider groups that might meet for specific hobbies.",
#     "Think about both active and relaxed group settings.",
#     "Use names that include popular slang or phrases.",
#     "Incorporate recent trends in naming conventions.",
#     "Generate some groups based on personality types.",
#     "Consider historical and futuristic group themes.",
#     "Include groups related to lifestyle and mindfulness.",
#     "Think of names for book clubs and media fans.",
#     "Include groups for gaming, technology, and science.",
#     "Think of groups for pet lovers and animal advocates.",
#     "Consider interest groups for different age groups.",
#     "Use names that might appeal to specific cultures.",
#     "Consider names that evoke a sense of adventure.",
#     "Include both serious and humorous group names.",
#     "Think about groups for fitness and sports fans.",
#     "Consider names for people interested in learning.",
#     "Include names for groups that focus on relaxation.",
# ]

# # Global list to hold all query results
# all_squad_data = []

# def save_batch_to_jsonl(data, filename):
#     """
#     Save a batch of squad name data to a JSONL file incrementally.
#     """
#     with open(filename, "a") as f:
#         for entry in data:
#             jsonl_entry = {
#                 "prompt": entry["prompt"],
#                 "completion": ", ".join(entry["squad_names"]),
#             }
#             f.write(json.dumps(jsonl_entry) + "\n")
#     print(f"Batch of {len(data)} squad names saved to {filename}")


# def generate_squad_names_from_summary(summary_text, target_count=names_per_prompt):
#     """
#     Generate squad names using the OpenAI client based on a given summary text.
#     """
#     generated_squads = set()
#     while len(generated_squads) < target_count:
#         random_suffix = random.choice(suffix_list)
#         try:
#             completion = client.chat.completions.create(
#                 model="gpt-4",
#                 messages=[
#                     {"role": "system", "content": "You are a helpful assistant for creating squad names."},
#                     {
#                         "role": "user",
#                         "content": (
#                             f"Based on the following description, suggest up to {batch_size} unique names for interest groups "
#                             f"or squads of people with shared interests:\n\n{summary_text}\n\n"
#                             "Generate names that include a variety of interests and naturally occurring groups, "
#                             "such as 'Athletes Squad,' 'Music Lovers Community,' 'Debate Club Enthusiasts,' 'Environmental Activists,' "
#                             "and 'Tech Innovators.' Be creative and make sure each name sounds like a natural group. " + random_suffix
#                         ),
#                     },
#                 ],
#             )
#             response_text = completion.choices[0].message.content.strip()
#             squad_names = response_text.split("\n")
#             generated_squads.update([name.strip() for name in squad_names if name.strip()])
#             print(f"Generated {len(generated_squads)} unique squads so far.")
#             time.sleep(1)  # Respect API limits

#         except openai.error.OpenAIError as e:
#             print(f"Error generating squad names: {e}")
#             break

#     return list(generated_squads)[:target_count]


# def generate_squads_from_prompts(prompts, start_index=1):
#     """
#     Generate squad names for each prompt starting from the specified index.
#     """
#     global all_squad_data
#     total_generated = 0

#     for idx, prompt in enumerate(prompts[start_index:], start=start_index):
#         node_name = prompt.get("node_name", "Unknown Node")
#         api_query_child_name = prompt.get("api_query_child_name", "Unknown Query")
#         prompt_content = prompt["prompt_extracted"]

#         summary_text = (
#             f"{prompt_content['these are details about the following subject: ' + node_name]['Summary']}\n\n"
#             f"Additional details:\n"
#             f"Topic: {prompt_content['these are details about the following subject: ' + node_name]['Topic']}\n"
#             f"Subtopics: {prompt_content['these are details about the following subject: ' + node_name]['Subtopics']}"
#         )

#         print(f"Generating squad names for {node_name} - {api_query_child_name}")

#         squad_names = generate_squad_names_from_summary(summary_text)

#         batch_entry = {
#             "node_name": node_name,
#             "api_query_child_name": api_query_child_name,
#             "prompt": summary_text,
#             "squad_names": squad_names,
#         }

#         all_squad_data.append(batch_entry)
#         total_generated += len(squad_names)

#         # Save the batch incrementally
#         save_batch_to_jsonl([batch_entry], output_file)

#         print(f"Processed prompt {idx + 1}/{len(prompts)} - Total squads generated so far: {total_generated}")

#     print("\nSquad name generation completed.")
#     print(f"Total squad names generated: {total_generated}")


# # Example run starting from the 11th item in prompts
# generate_squads_from_prompts(generated_prompts, start_index=10)


In [23]:
# import openai
# import time
# import random
# import json

# # Initialize OpenAI client
# client = openai.OpenAI()

# # Define global constants and variables
# output_file = "squad_names_training_data.jsonl"  # Incremental saving
# batch_size = 500  # Moderate batch size for API requests
# names_per_prompt = 2500  # Total squad names to generate per prompt
# suffix_list = [  # Diverse suffixes for prompt variation
#     "Include as many types as possible.",
#     "Think creatively and avoid repetition.",
#     "Consider various types of people groups.",
#     "Cover a diverse range of interests.",
#     "Use different community styles and languages.",
#     "Be inclusive of all kinds of interests.",
#     "Generate some unexpected group names.",
#     "Think of unique groups and subcultures.",
#     "Avoid generic terms; make names memorable.",
#     "Include academic, recreational, and cultural squads.",
#     "Consider groups with both local and global appeal.",
#     "Use both formal and informal styles of naming.",
#     "Include both niche and mainstream interests.",
#     "Think of names that appeal to different ages.",
#     "Include names that would fit specific settings.",
#     "Focus on creative and new community names.",
#     "Generate group names that sound modern.",
#     "Use some names that include numbers or symbols.",
#     "Consider groups based on new or emerging trends.",
#     "Think of groups that connect online and offline.",
#     "Suggest names for both large and small groups.",
#     "Consider local, national, and global communities.",
#     "Think about groups with seasonal interests.",
#     "Use names that reflect popular events or festivals.",
#     "Consider student clubs and academic groups.",
#     "Include names for different professional groups.",
#     "Think of environmental, tech, and arts-based groups.",
#     "Use quirky and fun names for casual gatherings.",
#     "Include groups for both indoor and outdoor activities.",
#     "Think of names that involve social media trends.",
#     "Consider groups with culinary or travel interests.",
#     "Use names that appeal to niche online communities.",
#     "Include groups with philosophical or wellness themes.",
#     "Consider groups that might meet for specific hobbies.",
#     "Think about both active and relaxed group settings.",
#     "Use names that include popular slang or phrases.",
#     "Incorporate recent trends in naming conventions.",
#     "Generate some groups based on personality types.",
#     "Consider historical and futuristic group themes.",
#     "Include groups related to lifestyle and mindfulness.",
#     "Think of names for book clubs and media fans.",
#     "Include groups for gaming, technology, and science.",
#     "Think of groups for pet lovers and animal advocates.",
#     "Consider interest groups for different age groups.",
#     "Use names that might appeal to specific cultures.",
#     "Consider names that evoke a sense of adventure.",
#     "Include both serious and humorous group names.",
#     "Think about groups for fitness and sports fans.",
#     "Consider names for people interested in learning.",
#     "Include names for groups that focus on relaxation.",
# ]

# # Global list to hold all query results
# new_all_squad_data = []

# def save_batch_to_jsonl(data, filename):
#     """
#     Save a batch of squad name data to a JSONL file incrementally.
#     """
#     with open(filename, "a") as f:
#         for entry in data:
#             jsonl_entry = {
#                 "prompt": entry["prompt"],
#                 "completion": ", ".join(entry["squad_names"]),
#             }
#             f.write(json.dumps(jsonl_entry) + "\n")
#     print(f"Batch of {len(data)} squad names saved to {filename}")


# def generate_squad_names_from_summary(summary_text, target_count=names_per_prompt):
#     """
#     Generate squad names using the OpenAI client based on a given summary text.
#     """
#     generated_squads = set()
#     while len(generated_squads) < target_count:
#         random_suffix = random.choice(suffix_list)
#         try:
#             completion = client.chat.completions.create(
#                 model="gpt-4",
#                 messages=[
#                     {"role": "system", "content": "You are a helpful assistant for creating squad names."},
#                     {
#                         "role": "user",
#                         "content": (
#                             f"Based on the following description, suggest up to {batch_size} unique names for interest groups "
#                             f"or squads of people with shared interests:\n\n{summary_text}\n\n"
#                             "Generate names that include a variety of interests and naturally occurring groups, "
#                             "such as 'Athletes Squad,' 'Music Lovers Community,' 'Debate Club Enthusiasts,' 'Environmental Activists,' "
#                             "and 'Tech Innovators.' Be creative and make sure each name sounds like a natural group. " + random_suffix
#                         ),
#                     },
#                 ],
#             )
#             response_text = completion.choices[0].message.content.strip()
#             squad_names = response_text.split("\n")
#             generated_squads.update([name.strip() for name in squad_names if name.strip()])
#             print(f"Generated {len(generated_squads)} unique squads so far.")
#             time.sleep(1)  # Respect API limits

#         except openai.error.OpenAIError as e:
#             print(f"Error generating squad names: {e}")
#             break

#     return list(generated_squads)[:target_count]


# def generate_squads_from_prompts(prompts, start_index=32):
#     """
#     Generate squad names for each prompt starting from the specified index.
#     """
#     global new_all_squad_data
#     total_generated = 0

#     for idx, prompt in enumerate(prompts[start_index:], start=start_index):
#         node_name = prompt.get("node_name", "Unknown Node")
#         api_query_child_name = prompt.get("api_query_child_name", "Unknown Query")
#         prompt_content = prompt["prompt_extracted"]

#         summary_text = (
#             f"{prompt_content['these are details about the following subject: ' + node_name]['Summary']}\n\n"
#             f"Additional details:\n"
#             f"Topic: {prompt_content['these are details about the following subject: ' + node_name]['Topic']}\n"
#             f"Subtopics: {prompt_content['these are details about the following subject: ' + node_name]['Subtopics']}"
#         )

#         print(f"Generating squad names for {node_name} - {api_query_child_name}")

#         squad_names = generate_squad_names_from_summary(summary_text)

#         batch_entry = {
#             "node_name": node_name,
#             "api_query_child_name": api_query_child_name,
#             "prompt": summary_text,
#             "squad_names": squad_names,
#         }

#         all_squad_data.append(batch_entry)
#         total_generated += len(squad_names)

#         # Save the batch incrementally
#         save_batch_to_jsonl([batch_entry], output_file)

#         print(f"Processed prompt {idx + 1}/{len(prompts)} - Total squads generated so far: {total_generated}")

#     print("\nSquad name generation completed.")
#     print(f"Total squad names generated: {total_generated}")


# # Example run starting from the 11th item in prompts
# generate_squads_from_prompts(generated_prompts, start_index=10)


In [24]:
import openai
import time
import random
import json

# Initialize OpenAI client
client = openai.OpenAI()

# Define global constants and variables
output_file = "squad_names_training_data.jsonl"  # Incremental saving
batch_size = 500  # Moderate batch size for API requests
names_per_prompt = 2500  # Total squad names to generate per prompt
suffix_list = [  # Diverse suffixes for prompt variation
    "Include as many types as possible.",
    "Think creatively and avoid repetition.",
    "Consider various types of people groups.",
    "Cover a diverse range of interests.",
    "Use different community styles and languages.",
    "Be inclusive of all kinds of interests.",
    "Generate some unexpected group names.",
    "Think of unique groups and subcultures.",
    "Avoid generic terms; make names memorable.",
    "Include academic, recreational, and cultural squads.",
    "Consider groups with both local and global appeal.",
    "Use both formal and informal styles of naming.",
    "Include both niche and mainstream interests.",
    "Think of names that appeal to different ages.",
    "Include names that would fit specific settings.",
    "Focus on creative and new community names.",
    "Generate group names that sound modern.",
    "Use some names that include numbers or symbols.",
    "Consider groups based on new or emerging trends.",
    "Think of groups that connect online and offline.",
    "Suggest names for both large and small groups.",
    "Consider local, national, and global communities.",
    "Think about groups with seasonal interests.",
    "Use names that reflect popular events or festivals.",
    "Consider student clubs and academic groups.",
    "Include names for different professional groups.",
    "Think of environmental, tech, and arts-based groups.",
    "Use quirky and fun names for casual gatherings.",
    "Include groups for both indoor and outdoor activities.",
    "Think of names that involve social media trends.",
    "Consider groups with culinary or travel interests.",
    "Use names that appeal to niche online communities.",
    "Include groups with philosophical or wellness themes.",
    "Consider groups that might meet for specific hobbies.",
    "Think about both active and relaxed group settings.",
    "Use names that include popular slang or phrases.",
    "Incorporate recent trends in naming conventions.",
    "Generate some groups based on personality types.",
    "Consider historical and futuristic group themes.",
    "Include groups related to lifestyle and mindfulness.",
    "Think of names for book clubs and media fans.",
    "Include groups for gaming, technology, and science.",
    "Think of groups for pet lovers and animal advocates.",
    "Consider interest groups for different age groups.",
    "Use names that might appeal to specific cultures.",
    "Consider names that evoke a sense of adventure.",
    "Include both serious and humorous group names.",
    "Think about groups for fitness and sports fans.",
    "Consider names for people interested in learning.",
    "Include names for groups that focus on relaxation.",
]

# Global list to hold all query results
new_batch_all_squad_data = []

def save_batch_to_jsonl(data, filename):
    """
    Save a batch of squad name data to a JSONL file incrementally.
    """
    with open(filename, "a") as f:
        for entry in data:
            jsonl_entry = {
                "prompt": entry["prompt"],
                "completion": ", ".join(entry["squad_names"]),
            }
            f.write(json.dumps(jsonl_entry) + "\n")
    print(f"Batch of {len(data)} squad names saved to {filename}")


def generate_squad_names_from_summary(summary_text, target_count=names_per_prompt):
    """
    Generate squad names using the OpenAI client based on a given summary text.
    """
    generated_squads = set()
    while len(generated_squads) < target_count:
        random_suffix = random.choice(suffix_list)
        try:
            completion = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant for creating squad names."},
                    {
                        "role": "user",
                        "content": (
                            f"Based on the following description, suggest up to {batch_size} unique names for interest groups "
                            f"or squads of people with shared interests:\n\n{summary_text}\n\n"
                            "Generate names that include a variety of interests and naturally occurring groups, "
                            "such as 'Athletes Squad,' 'Music Lovers Community,' 'Debate Club Enthusiasts,' 'Environmental Activists,' "
                            "and 'Tech Innovators.' Be creative and make sure each name sounds like a natural group. " + random_suffix
                        ),
                    },
                ],
            )
            response_text = completion.choices[0].message.content.strip()
            squad_names = response_text.split("\n")
            generated_squads.update([name.strip() for name in squad_names if name.strip()])
            print(f"Generated {len(generated_squads)} unique squads so far.")
            time.sleep(1)  # Respect API limits

        except openai.error.OpenAIError as e:
            print(f"Error generating squad names: {e}")
            break

    return list(generated_squads)[:target_count]


def generate_squads_from_prompts(prompts, start_index=77):
    """
    Generate squad names for each prompt starting from the specified index.
    """
    global new_batch_all_squad_data
    total_generated = 0

    for idx, prompt in enumerate(prompts[start_index:], start=start_index):
        node_name = prompt.get("node_name", "Unknown Node")
        api_query_child_name = prompt.get("api_query_child_name", "Unknown Query")
        prompt_content = prompt["prompt_extracted"]

        summary_text = (
            f"{prompt_content['these are details about the following subject: ' + node_name]['Summary']}\n\n"
            f"Additional details:\n"
            f"Topic: {prompt_content['these are details about the following subject: ' + node_name]['Topic']}\n"
            f"Subtopics: {prompt_content['these are details about the following subject: ' + node_name]['Subtopics']}"
        )

        print(f"Generating squad names for {node_name} - {api_query_child_name}")

        squad_names = generate_squad_names_from_summary(summary_text)

        batch_entry = {
            "node_name": node_name,
            "api_query_child_name": api_query_child_name,
            "prompt": summary_text,
            "squad_names": squad_names,
        }

        new_batch_all_squad_data.append(batch_entry)
        total_generated += len(squad_names)

        # Save the batch incrementally
        save_batch_to_jsonl([batch_entry], output_file)

        print(f"Processed prompt {idx + 1}/{len(prompts)} - Total squads generated so far: {total_generated}")

    print("\nSquad name generation completed.")
    print(f"Total squad names generated: {total_generated}")


# Example run starting from the 11th item in prompts
generate_squads_from_prompts(generated_prompts, start_index=10)


Generating squad names for Movies and TV Shows - TV Shows
Generated 100 unique squads so far.
Generated 600 unique squads so far.
Generated 1098 unique squads so far.
Generated 1248 unique squads so far.
Generated 1348 unique squads so far.
Generated 1848 unique squads so far.
Generated 1950 unique squads so far.
Generated 2450 unique squads so far.
Generated 2950 unique squads so far.
Batch of 1 squad names saved to squad_names_training_data.jsonl
Processed prompt 11/201 - Total squads generated so far: 2500
Generating squad names for Movies and TV Shows - Movies
Generated 500 unique squads so far.
Generated 1000 unique squads so far.
Generated 1100 unique squads so far.
Generated 1600 unique squads so far.
Generated 2099 unique squads so far.
Generated 2159 unique squads so far.
Generated 2659 unique squads so far.
Batch of 1 squad names saved to squad_names_training_data.jsonl
Processed prompt 12/201 - Total squads generated so far: 5000
Generating squad names for Movies and TV Show

AttributeError: module 'openai' has no attribute 'error'